[Google Vision python docs](https://googleapis.dev/python/vision/latest/index.html)

cf. [another tutorial](https://cloud.google.com/functions/docs/tutorials/ocr), which uses cloud functions

It turns out, google OCR will work with PDFs only if thy are stored in google cloud. Since that just adds another layer of complexity, I'm going to work with Backblaze

In [87]:
from PIL import ImageDraw
from google.oauth2 import service_account
creds_path = '/home/src/oed/creds.json'

creds = service_account.Credentials.from_service_account_file(creds_path)

In [88]:
%connect_info

{
  "shell_port": 42625,
  "iopub_port": 34709,
  "stdin_port": 34463,
  "control_port": 38627,
  "hb_port": 32969,
  "ip": "127.0.0.1",
  "key": "fc881614-4166bf89ba71261b08a7a703",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-af21ada3-6f4e-430b-8015-e238442182df.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [89]:
from google.protobuf.json_format import MessageToDict
from google.cloud import vision
client = vision.ImageAnnotatorClient(credentials=creds)


feature = vision.Feature(
    type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)
# async_request = vision.AsyncAnnotateFileRequest(
#     features=[feature], input_config=input_config,
#     output_config=output_config)

source_uri = 'https://jeroen.github.io/images/testocr.png'
# source_uri = 'https://africau.edu/images/default/sample.pdf'

source_uri = 'https://ohuiginn.net/tmp/singlepage-1.png'
image_src = vision.ImageSource(image_uri = source_uri)
img = vision.Image(source=image_src)
page_as_protobuf = client.annotate_image({
    'image': img,

# 'image': {'source': {'image_uri': source_uri}},
    'features': [feature]
})
page_as_protobuf
page_as_dict = MessageToDict(page_as_protobuf._pb)

In [90]:
# cargo-cult from https://towardsdatascience.com/how-to-extract-the-text-from-pdfs-using-python-and-the-google-cloud-vision-api-7a0a798adc13


# Display with bounding boxes
# from https://cloud.google.com/vision/docs/fulltext-annotations
import argparse
from enum import Enum
import io

from google.cloud import vision
from PIL import Image, ImageDraw


class FeatureType(Enum):
    PAGE = 1
    BLOCK = 2
    PARA = 3
    WORD = 4
    SYMBOL = 5


def draw_boxes(image, bounds, color):
    """Draw a border around the image using the hints in the vector list."""
    draw = ImageDraw.Draw(image)

    for bound in bounds:
        draw.polygon([
            bound.vertices[0].x, bound.vertices[0].y,
            bound.vertices[1].x, bound.vertices[1].y,
            bound.vertices[2].x, bound.vertices[2].y,
            bound.vertices[3].x, bound.vertices[3].y], None, color)
    return image


def get_document_bounds(image_file, feature):
    """Returns document bounds given an image."""
    client = vision.ImageAnnotatorClient(credentials=creds)

    bounds = []

    with io.open(image_file, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.document_text_detection(image=image)
    document = response.full_text_annotation

    # Collect specified feature bounds by enumerating all document features
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    for symbol in word.symbols:
                        if (feature == FeatureType.SYMBOL):
                            bounds.append(symbol.bounding_box)

                    if (feature == FeatureType.WORD):
                        bounds.append(word.bounding_box)

                if (feature == FeatureType.PARA):
                    bounds.append(paragraph.bounding_box)

            if (feature == FeatureType.BLOCK):
                bounds.append(block.bounding_box)

    # The list `bounds` contains the coordinates of the bounding boxes.
    return bounds


def render_doc_text(filein, fileout):
    image = Image.open(filein)
    bounds = get_document_bounds(filein, FeatureType.BLOCK)
    draw_boxes(image, bounds, 'blue')
    bounds = get_document_bounds(filein, FeatureType.PARA)
    draw_boxes(image, bounds, 'red')
    bounds = get_document_bounds(filein, FeatureType.WORD)
    draw_boxes(image, bounds, 'yellow')

    if fileout != 0:
        image.save(fileout)
    else:
        image.show()

def display_annotation():
    file_in = './data/singlepage-1.png'
    file_out = '/tmp/singlepage-1-annotated.png'
    render_doc_text(file_in, file_out)

display_annotation()

coordinates are (0,0) in top right